In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_pose = mp.solutions.pose # Mediapipe Solutions

In [11]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(static_image_mode=False,
                          model_complexity=1,
                          smooth_landmarks=True,
                          enable_segmentation=False,
                          smooth_segmentation=True,
                          min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

 # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [8]:
cap = cv2.VideoCapture(0)
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps=cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
videowriter = cv2.VideoWriter('deadlift.avi', fourcc, fps, (int(w),int(h)))

while cap.isOpened():
    ret, frame = cap.read()
    videowriter.write(frame)
    cv2.imshow('video feed', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
videowriter.release()
cv2.destroyAllWindows()

In [6]:
cap.release()
videowriter.release()
cv2.destroyAllWindows()

In [9]:
import csv
import os

In [4]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [44]:
with open('points.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [33]:
def export_landmarks(results,action):        
    try:
        # Extract Pose landmarks
        pose = results.pose_landmarks.landmark
        points = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

        # Append class name
        points.insert(0, action)

        # Export to CSV
        with open('points.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(points)

    except :
        pass

In [53]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(static_image_mode=False,
                          model_complexity=1,
                          smooth_landmarks=True,
                          enable_segmentation=False,
                          smooth_segmentation=True,
                          min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

 # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        k=cv2.waitKey(1)
        if k==97:
            export_landmarks(results,'up')
        if k==122:
            export_landmarks(results,'down')
            
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [22]:
cap.release()
cv2.destroyAllWindows()

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('points.csv')
X = df.drop('class', axis=1).values # features
y = df['class'] # target value
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [4]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [5]:
from sklearn.metrics import accuracy_score, recall_score,precision_score # Accuracy metrics 
import pickle 
for algorithm, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algorithm, accuracy_score(y_test, yhat),
          recall_score(y_test, yhat,average='binary',pos_label='up'),
          precision_score(y_test, yhat,average='binary',pos_label='up'))

lr 0.8333333333333334 1.0 0.75
rc 1.0 1.0 1.0
rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [49]:
with open('deadlift.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [6]:
with open('deadlift.pkl', 'rb') as f:
    model=pickle.load(f)

In [26]:
cap = cv2.VideoCapture(0)
counter=0
with mp_pose.Pose(static_image_mode=False,
                          model_complexity=1,
                          smooth_landmarks=True,
                          enable_segmentation=False,
                          smooth_segmentation=True,
                          min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

 # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        try:
            # Extract Pose landmarks
            pose_raw = results.pose_landmarks.landmark
            points = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_raw]).flatten())
            X = pd.DataFrame([points])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            cv2.putText(image, body_language_class, (50,50), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2)), (100,100), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)    
            cv2.imshow('Raw Webcam Feed', image)
            if body_language_class=='down' and body_language_prob[body_language_prob.argmax()]>.5:
                current_state='down'
            elif  current_state='down' and body_language_class=='up' and body_language_prob[body_language_prob.argmax()]>.5:
                current_state='up'
                counter+=1
            cv2.putText(image, str(counter), (150,150), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        except :
            pass


        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

SyntaxError: invalid syntax (2262590218.py, line 42)